# Analysis of cell-free synthetic gene expression

In [ ]:
from flapjack import Flapjack, layout_print
import numpy as np
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as io
import pandas as pd
import getpass
%matplotlib inline

## Computing and plotting reaction velocity
Reaction velocity is the first derivative of fluorescence (a time series). It can be plotted by selecting "Velocity" from the analysis dropdown. First select the cell-free study in the filter pane. Now you will see below a list of DNAs in the study. First select L1_23A and L1_24A (the YFP expressing plasmids). Select groupby Name, Media, DNA. Select averaging "Mean +/- std" and normalization "Mean/std" and then click "Analyse". Selecting the YFP tab you can see the velocity profiles for each DNA in each cell-free extract batch (media). Similarly, you can plot the CFP expressing DNAs by selecting L1_29A and L1_30A.

We can see that the dynamics of each gene are quite similar, but do appear to vary from batch to batch. Here we normalized the traces so we cannot see magnitude differences. In the next section we analyse the variation in magnitude of velocity.

In [ ]:
user = input()
passwd = getpass.getpass()
fj = Flapjack('3.128.232.8:8000')
fj.log_in(username=user, password=passwd)

## Analysing reaction velocity
To analyse the magnitude of gene expression, we can get the maximum reaction velocity over each time series from the Flapjack API:

In [ ]:
study = fj.get('study', name='CFPS of CFP YFP variants')
df = fj.analysis(study=study.id,
                 type='Max Velocity',
                 pre_smoothing=11,
                 post_smoothing=11
                )

Now we can group this dataframe to make a bar graph showing the maximum velocity in each of the combinations of extract batch and DNA that were tested:

In [ ]:
cols = {
    'E1': '#1f77b4', 
    'E2': '#ff7f0e', 
    'E3': '#2ca02c'
}
subplot_titles = ['CFP', 'YFP']
fig = make_subplots(rows=2, 
                    cols=1, 
                    vertical_spacing=0.15, 
                    horizontal_spacing=0.1,
                    subplot_titles=subplot_titles
                   )
row = 1
for name,name_data in df.groupby('Signal'):
    print(name)
    for media,media_data in name_data.groupby('Media'):
        grouped_dna = media_data.groupby('Vector')
        x = [dna for dna,data in grouped_dna]
        y = [data['Velocity'].mean() for dna,data in grouped_dna]        
        err_y = [data['Velocity'].std() for dna,data in grouped_dna]        
        error_y = dict(type='data', # value of error bar given in data coordinates
                        array=err_y,
                        visible=True)
        bar = go.Bar(x=x, 
                     y=y,
                     error_y=error_y, 
                     name='Batch '+media, 
                     legendgroup=media,
                     marker=dict(color=cols[media], line=dict(color='rgb(0,0,0)', width=1.)),
                     showlegend=row==1
                    )

        fig.add_trace(bar, row=row, col=1)
    row += 1

fig.update_yaxes(title_text='Max. velocity', col=1)
fig = layout_print(fig, width=3.3, height=3.3, font_size=6.)
fig.update_layout(barmode='group')
fig.show()
    

In [ ]:
df[df.Signal=='YFP'].groupby(['Vector', 'Media']).mean().plot.bar(y='Velocity')